In [ ]:
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, f1_score, accuracy_score, precision_score, recall_score
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define the path to the directory containing the feature files
directory_path = '/content/drive/MyDrive/FYP_dataset/features'

# Get a list of all feature file paths in the directory
feature_files = glob.glob(directory_path + '/*.csv')

# Read the feature file into a DataFrame
df = pd.read_csv(feature_files[0])

list_of_column_names = list(df.columns.drop(['Arousal_Value', 'Valence_Value', 'frameTime']))

print(list_of_column_names)
print(len(list_of_column_names))

['F0final_sma_stddev', 'F0final_sma_amean', 'voicingFinalUnclipped_sma_stddev', 'voicingFinalUnclipped_sma_amean', 'jitterLocal_sma_stddev', 'jitterLocal_sma_amean', 'jitterDDP_sma_stddev', 'jitterDDP_sma_amean', 'shimmerLocal_sma_stddev', 'shimmerLocal_sma_amean', 'logHNR_sma_stddev', 'logHNR_sma_amean', 'audspec_lengthL1norm_sma_stddev', 'audspec_lengthL1norm_sma_amean', 'audspecRasta_lengthL1norm_sma_stddev', 'audspecRasta_lengthL1norm_sma_amean', 'pcm_RMSenergy_sma_stddev', 'pcm_RMSenergy_sma_amean', 'pcm_zcr_sma_stddev', 'pcm_zcr_sma_amean', 'audSpec_Rfilt_sma[0]_stddev', 'audSpec_Rfilt_sma[0]_amean', 'audSpec_Rfilt_sma[1]_stddev', 'audSpec_Rfilt_sma[1]_amean', 'audSpec_Rfilt_sma[2]_stddev', 'audSpec_Rfilt_sma[2]_amean', 'audSpec_Rfilt_sma[3]_stddev', 'audSpec_Rfilt_sma[3]_amean', 'audSpec_Rfilt_sma[4]_stddev', 'audSpec_Rfilt_sma[4]_amean', 'audSpec_Rfilt_sma[5]_stddev', 'audSpec_Rfilt_sma[5]_amean', 'audSpec_Rfilt_sma[6]_stddev', 'audSpec_Rfilt_sma[6]_amean', 'audSpec_Rfilt_sma[7

In [ ]:
! pip install opensmile
import pandas as pd
import opensmile

# Initialize OpenSMILE
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
)

# Process the audio file
y = smile.process_file('/content/drive/MyDrive/FYP_dataset/CNN Features/abc.wav')

# Set Pandas options to display more rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Print the extracted features
smile_features= set((y.keys()))
print(smile_features)

{'audSpec_Rfilt_sma[25]_stddevFallingSlope', 'mfcc_sma[6]_segLenStddev', 'audSpec_Rfilt_sma[7]_peakMeanRel', 'audSpec_Rfilt_sma_de[15]_meanFallingSlope', 'audSpec_Rfilt_sma[18]_leftctime', 'audSpec_Rfilt_sma[9]_iqr1-2', 'mfcc_sma[10]_minSegLen', 'audspec_lengthL1norm_sma_lpgain', 'pcm_RMSenergy_sma_upleveltime50', 'audSpec_Rfilt_sma_de[10]_pctlrange0-1', 'pcm_fftMag_spectralKurtosis_sma_de_meanSegLen', 'mfcc_sma_de[11]_lpc3', 'pcm_fftMag_fband250-650_sma_peakRangeRel', 'audSpec_Rfilt_sma[22]_skewness', 'audSpec_Rfilt_sma_de[1]_maxPos', 'mfcc_sma_de[8]_stddevFallingSlope', 'mfcc_sma_de[3]_upleveltime90', 'mfcc_sma[14]_upleveltime25', 'pcm_fftMag_spectralVariance_sma_segLenStddev', 'voicingFinalUnclipped_sma_lpc4', 'audSpec_Rfilt_sma[23]_peakMeanAbs', 'audSpec_Rfilt_sma_de[19]_lpc3', 'audSpec_Rfilt_sma[10]_rqmean', 'mfcc_sma_de[4]_quartile3', 'audSpec_Rfilt_sma_de[8]_percentile1.0', 'mfcc_sma[10]_minPos', 'pcm_fftMag_fband250-650_sma_linregerrQ', 'mfcc_sma[11]_iqr1-2', 'audSpec_Rfilt_sma

In [ ]:
set_our_features=set(list_of_column_names)
print(set_our_features)

{'pcm_fftMag_spectralRollOff25.0_sma_stddev', 'audSpec_Rfilt_sma[0]_stddev', 'audSpec_Rfilt_sma_de[6]_amean', 'pcm_RMSenergy_sma_stddev', 'audSpec_Rfilt_sma_de[14]_stddev', 'pcm_fftMag_mfcc_sma[3]_stddev', 'pcm_fftMag_spectralRollOff90.0_sma_de_amean', 'audSpec_Rfilt_sma[21]_amean', 'audSpec_Rfilt_sma[9]_stddev', 'pcm_fftMag_spectralSkewness_sma_stddev', 'pcm_fftMag_mfcc_sma_de[4]_amean', 'audSpec_Rfilt_sma_de[25]_stddev', 'audspec_lengthL1norm_sma_amean', 'pcm_fftMag_mfcc_sma[8]_amean', 'pcm_fftMag_mfcc_sma[12]_amean', 'audSpec_Rfilt_sma_de[1]_stddev', 'audSpec_Rfilt_sma[16]_stddev', 'pcm_fftMag_spectralRollOff50.0_sma_de_stddev', 'pcm_fftMag_spectralHarmonicity_sma_stddev', 'audSpec_Rfilt_sma_de[24]_amean', 'pcm_fftMag_psySharpness_sma_stddev', 'pcm_fftMag_spectralFlux_sma_stddev', 'audSpec_Rfilt_sma[24]_stddev', 'audSpec_Rfilt_sma_de[2]_amean', 'pcm_fftMag_mfcc_sma[10]_stddev', 'logHNR_sma_de_stddev', 'pcm_fftMag_mfcc_sma[12]_stddev', 'pcm_fftMag_mfcc_sma[14]_amean', 'pcm_fftMag_mfc

In [ ]:
dont_have_features_set=set_our_features-smile_features
print(dont_have_features_set)

dont_have_features_list=list(dont_have_features_set)
print(len(dont_have_features_list))

{'audSpec_Rfilt_sma_de[6]_amean', 'pcm_fftMag_mfcc_sma[3]_stddev', 'pcm_fftMag_spectralRollOff90.0_sma_de_amean', 'pcm_fftMag_mfcc_sma_de[4]_amean', 'pcm_fftMag_mfcc_sma[8]_amean', 'pcm_fftMag_mfcc_sma[12]_amean', 'audSpec_Rfilt_sma_de[24]_amean', 'audSpec_Rfilt_sma_de[2]_amean', 'audSpec_Rfilt_sma_de[14]_amean', 'pcm_fftMag_mfcc_sma[10]_stddev', 'pcm_fftMag_mfcc_sma[12]_stddev', 'pcm_fftMag_mfcc_sma[14]_amean', 'pcm_fftMag_mfcc_sma[9]_amean', 'pcm_fftMag_mfcc_sma_de[7]_stddev', 'audSpec_Rfilt_sma_de[18]_amean', 'pcm_fftMag_spectralSkewness_sma_de_amean', 'pcm_fftMag_mfcc_sma[2]_amean', 'pcm_fftMag_mfcc_sma_de[12]_amean', 'audSpec_Rfilt_sma_de[20]_amean', 'audSpec_Rfilt_sma_de[17]_amean', 'pcm_fftMag_mfcc_sma_de[8]_stddev', 'audSpec_Rfilt_sma_de[1]_amean', 'pcm_fftMag_mfcc_sma[6]_amean', 'pcm_fftMag_mfcc_sma_de[4]_stddev', 'pcm_fftMag_mfcc_sma_de[13]_amean', 'pcm_fftMag_mfcc_sma[4]_amean', 'pcm_fftMag_mfcc_sma[5]_stddev', 'pcm_fftMag_mfcc_sma_de[14]_amean', 'pcm_fftMag_mfcc_sma_de[11]_

In [ ]:
smile_features_have_in_the_dataset=set_our_features - dont_have_features_set
print(smile_features_have_in_the_dataset)

smile_features_have_in_the_dataset=list(smile_features_have_in_the_dataset)
print(len(smile_features_have_in_the_dataset))

{'pcm_fftMag_spectralRollOff25.0_sma_stddev', 'audSpec_Rfilt_sma[0]_stddev', 'audSpec_Rfilt_sma_de[14]_stddev', 'pcm_RMSenergy_sma_stddev', 'audSpec_Rfilt_sma[21]_amean', 'audSpec_Rfilt_sma[9]_stddev', 'pcm_fftMag_spectralSkewness_sma_stddev', 'audSpec_Rfilt_sma_de[25]_stddev', 'audspec_lengthL1norm_sma_amean', 'audSpec_Rfilt_sma_de[1]_stddev', 'audSpec_Rfilt_sma[16]_stddev', 'pcm_fftMag_spectralRollOff50.0_sma_de_stddev', 'pcm_fftMag_spectralHarmonicity_sma_stddev', 'pcm_fftMag_psySharpness_sma_stddev', 'pcm_fftMag_spectralFlux_sma_stddev', 'audSpec_Rfilt_sma[24]_stddev', 'logHNR_sma_de_stddev', 'audSpec_Rfilt_sma[19]_amean', 'pcm_fftMag_spectralHarmonicity_sma_amean', 'shimmerLocal_sma_de_amean', 'pcm_RMSenergy_sma_amean', 'audSpec_Rfilt_sma[14]_amean', 'audSpec_Rfilt_sma[12]_amean', 'audSpec_Rfilt_sma[20]_stddev', 'audSpec_Rfilt_sma[6]_stddev', 'audspecRasta_lengthL1norm_sma_stddev', 'pcm_fftMag_psySharpness_sma_de_stddev', 'voicingFinalUnclipped_sma_de_amean', 'jitterDDP_sma_de_ame

In [ ]:
drop_list=[]
#drop_list=dont_have_features_list
for i in range(0,len(dont_have_features_list)):
  drop_list.append(dont_have_features_list[i])


In [ ]:
drop_list.append('Arousal_Value')
drop_list.append('Valence_Value')
drop_list.append('frameTime')

print(drop_list)
print(len(drop_list))

['audSpec_Rfilt_sma_de[6]_amean', 'pcm_fftMag_mfcc_sma[3]_stddev', 'pcm_fftMag_spectralRollOff90.0_sma_de_amean', 'pcm_fftMag_mfcc_sma_de[4]_amean', 'pcm_fftMag_mfcc_sma[8]_amean', 'pcm_fftMag_mfcc_sma[12]_amean', 'audSpec_Rfilt_sma_de[24]_amean', 'audSpec_Rfilt_sma_de[2]_amean', 'audSpec_Rfilt_sma_de[14]_amean', 'pcm_fftMag_mfcc_sma[10]_stddev', 'pcm_fftMag_mfcc_sma[12]_stddev', 'pcm_fftMag_mfcc_sma[14]_amean', 'pcm_fftMag_mfcc_sma[9]_amean', 'pcm_fftMag_mfcc_sma_de[7]_stddev', 'audSpec_Rfilt_sma_de[18]_amean', 'pcm_fftMag_spectralSkewness_sma_de_amean', 'pcm_fftMag_mfcc_sma[2]_amean', 'pcm_fftMag_mfcc_sma_de[12]_amean', 'audSpec_Rfilt_sma_de[20]_amean', 'audSpec_Rfilt_sma_de[17]_amean', 'pcm_fftMag_mfcc_sma_de[8]_stddev', 'audSpec_Rfilt_sma_de[1]_amean', 'pcm_fftMag_mfcc_sma[6]_amean', 'pcm_fftMag_mfcc_sma_de[4]_stddev', 'pcm_fftMag_mfcc_sma_de[13]_amean', 'pcm_fftMag_mfcc_sma[4]_amean', 'pcm_fftMag_mfcc_sma[5]_stddev', 'pcm_fftMag_mfcc_sma_de[14]_amean', 'pcm_fftMag_mfcc_sma_de[11]_

In [ ]:
print(len(feature_files))

1802


In [ ]:
# Define lists to store the feature and target data
X = []
y_a = []
y_v = []

# Iterate over each feature file
for file in feature_files:
    # Read the feature file into a DataFrame
    df = pd.read_csv(file)

    # Extract the features and target variables
    features = df.drop(drop_list, axis=1).values
    arousal = df['Arousal_Value'].values
    valence = df['Valence_Value'].values

    # Append the data to the lists
    X.append(features)
    y_a.append(arousal)
    y_v.append(valence)

# Concatenate the feature and target arrays
X = np.concatenate(X)
y_a = np.concatenate(y_a)
y_v = np.concatenate(y_v)

# Scale the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Reshape the input data for LSTM
X_reshaped = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

# Define the number of folds for cross-validation
num_folds = 5

# Initialize lists to store evaluation results
mse_a = []
mae_a = []
rmse_a = []
r2_a = []
f1_a = []
accuracy_a = []
precision_a = []
recall_a = []

# Perform tenfold cross-validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X_reshaped):
    X_train, X_val = X_reshaped[train_index], X_reshaped[test_index]
    y_a_train, y_a_val = y_a[train_index], y_a[test_index]

    # Split the validation set further into val and test sets
    val_size = len(X_val) // 2
    X_val, X_test = X_val[:val_size], X_val[val_size:]
    y_a_val, y_a_test = y_a_val[:val_size], y_a_val[val_size:]

    # Build the BiLSTM model for Arousal
    model_a = Sequential()
    model_a.add(Bidirectional(CuDNNLSTM(64, return_sequences=True), input_shape=(X_train.shape[1], 1)))
    model_a.add(Flatten())

    # Add a DNN layer after the BiLSTM
    model_a.add(Dense(512, activation='relu'))

    model_a.add(Dense(1))
    model_a.compile(loss='mse', optimizer=Adam(learning_rate=0.001))

    # Train the Arousal model
    model_a.fit(X_train, y_a_train, validation_data=(X_val, y_a_val), epochs=25, batch_size=32)

    # Evaluate the model on the test set
    predictions_a = model_a.predict(X_test)

    # Reshape predictions_a to match the shape of y_a_test
    predictions_a = predictions_a.reshape(y_a_test.shape)

    # Calculate additional evaluation metrics
    mse_a.append(mean_squared_error(y_a_test, predictions_a))
    mae_a.append(mean_absolute_error(y_a_test, predictions_a))
    rmse_a.append(np.sqrt(mean_squared_error(y_a_test, predictions_a)))
    r2_a.append(r2_score(y_a_test, predictions_a))

    # Convert regression predictions to binary labels
    threshold = 0
    binary_predictions_a = (predictions_a >= threshold).astype(int)
    y_a_test_binary = (y_a_test >= threshold).astype(int)

    # Calculate F1-score, accuracy, precision, and recall for binary classification
    f1_a.append(f1_score(y_a_test_binary, binary_predictions_a))
    accuracy_a.append(accuracy_score(y_a_test_binary, binary_predictions_a))
    precision_a.append(precision_score(y_a_test_binary, binary_predictions_a))
    recall_a.append(recall_score(y_a_test_binary, binary_predictions_a))

# Calculate average performance across all folds
average_mse_a = np.mean(mse_a)
average_mae_a = np.mean(mae_a)
average_rmse_a = np.mean(rmse_a)
average_r2_a = np.mean(r2_a)
average_f1_a = np.mean(f1_a)
average_accuracy_a = np.mean(accuracy_a)
average_precision_a = np.mean(precision_a)
average_recall_a = np.mean(recall_a)

print(f'Average Arousal MSE: {average_mse_a:.4f}')
print(f'Average Arousal MAE: {average_mae_a:.4f}')
print(f'Average Arousal RMSE: {average_rmse_a:.4f}')
print(f'Average Arousal R2 Score: {average_r2_a:.4f}')
print(f'Average Arousal F1 Score: {average_f1_a:.4f}')
print(f'Average Arousal Accuracy: {average_accuracy_a:.4f}')
print(f'Average Arousal Precision: {average_precision_a:.4f}')
print(f'Average Arousal Recall: {average_recall_a:.4f}')

Epoch 1/25
2654/2654 [==============================] - 48s 15ms/step - loss: 0.0452 - val_loss: 0.0596
Epoch 2/25
2654/2654 [==============================] - 39s 15ms/step - loss: 0.0388 - val_loss: 0.0445
Epoch 3/25
2654/2654 [==============================] - 40s 15ms/step - loss: 0.0373 - val_loss: 0.0433
Epoch 4/25
2654/2654 [==============================] - 39s 15ms/step - loss: 0.0367 - val_loss: 0.0419
Epoch 5/25
2654/2654 [==============================] - 39s 15ms/step - loss: 0.0357 - val_loss: 0.0413
Epoch 6/25
2654/2654 [==============================] - 39s 15ms/step - loss: 0.0350 - val_loss: 0.0407
Epoch 7/25
2654/2654 [==============================] - 39s 15ms/step - loss: 0.0343 - val_loss: 0.0408
Epoch 8/25
2654/2654 [==============================] - 39s 15ms/step - loss: 0.0338 - val_loss: 0.0404
Epoch 9/25
2654/2654 [==============================] - 39s 15ms/step - loss: 0.0333 - val_loss: 0.0390
Epoch 10/25
2654/2654 [==============================] - 39s 15m